---

## 1️⃣ Setup & Verification

First, let's verify that all libraries are working correctly.

In [ ]:
# Import core libraries
import sys
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Image

# Quantum computing libraries
from qiskit import QuantumCircuit, Aer, execute, transpile
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_aer import AerSimulator

# Pretty printing
from rich import print as rprint
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

console = Console()

# Display welcome message
console.print(Panel.fit(
    "[bold green]✅ All libraries loaded successfully![/bold green]\n\n"
    "🎮 Houdinis Framework v1.0.0\n"
    "🔐 Quantum Cryptography Testing Platform\n\n"
    f"🐍 Python {sys.version.split()[0]}\n"
    "⚛️  Qiskit ready\n"
    "🖥️  Running in MyBinder.org",
    title="Houdinis Playground",
    border_style="green"
))

print("\n🎉 Ready to start! Run the next cell to create your first quantum circuit.")

---

## 2️⃣ Your First Quantum Circuit

Let's create a simple **Bell State** - one of the most fundamental quantum circuits that demonstrates **quantum entanglement**.

In [ ]:
# Create a quantum circuit with 2 qubits and 2 classical bits
qc = QuantumCircuit(2, 2)

# Apply Hadamard gate to the first qubit (creates superposition)
qc.h(0)

# Apply CNOT gate (creates entanglement)
qc.cx(0, 1)

# Measure both qubits
qc.measure([0, 1], [0, 1])

# Draw the circuit
print("\n📊 Circuit Diagram:\n")
print(qc.draw(output='text'))

console.print(Panel(
    "[bold cyan]What's happening here?[/bold cyan]\n\n"
    "• H gate: Creates superposition (50% |0⟩, 50% |1⟩)\n"
    "• CX gate: Entangles qubits (they become correlated)\n"
    "• Measure: Collapses quantum state to classical bits\n\n"
    "Result: You'll get either |00⟩ or |11⟩ (never |01⟩ or |10⟩)!",
    border_style="cyan"
))

---

## 3️⃣ Run on Quantum Simulator

Now let's execute this circuit 1024 times and see the results!

In [ ]:
# Get the quantum simulator backend
backend = Aer.get_backend('qasm_simulator')

# Execute the circuit with 1024 shots
job = execute(qc, backend, shots=1024)
result = job.result()
counts = result.get_counts(qc)

# Display results
console.print("\n[bold green]✅ Execution complete![/bold green]\n")

# Create results table
table = Table(title="Measurement Results (1024 shots)")
table.add_column("State", style="cyan", justify="center")
table.add_column("Count", style="magenta", justify="right")
table.add_column("Probability", style="green", justify="right")

for state, count in sorted(counts.items()):
    prob = count / 1024 * 100
    table.add_row(f"|{state}⟩", str(count), f"{prob:.1f}%")

console.print(table)

# Plot histogram
plt.figure(figsize=(10, 5))
plot_histogram(counts, figsize=(10, 5), color='skyblue')
plt.title('Bell State Measurement Results', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🎯 Notice: You only get |00⟩ and |11⟩ - this is quantum entanglement!")

---

## 4️⃣ Simple Quantum Attack: Grover's Search

Now let's try something more exciting - **Grover's Algorithm** for quantum search!

**Scenario:** Find a secret 2-bit number (00, 01, 10, or 11) using quantum speedup.

In [ ]:
def grover_oracle(qc, secret_number):
    """Oracle that marks the secret number"""
    # Convert number to binary (e.g., 3 -> '11')
    binary = format(secret_number, '02b')
    
    # Flip qubits that should be |0⟩
    for i, bit in enumerate(binary[::-1]):
        if bit == '0':
            qc.x(i)
    
    # Multi-controlled Z gate (marks the target)
    qc.cz(0, 1)
    
    # Flip back
    for i, bit in enumerate(binary[::-1]):
        if bit == '0':
            qc.x(i)

def grover_diffusion(qc):
    """Diffusion operator (amplifies amplitude of marked state)"""
    qc.h([0, 1])
    qc.x([0, 1])
    qc.cz(0, 1)
    qc.x([0, 1])
    qc.h([0, 1])

# Secret number to find (change this to test!)
secret = 2  # Binary: 10

# Create Grover's circuit
qc_grover = QuantumCircuit(2, 2)

# Initialize in superposition
qc_grover.h([0, 1])

# Apply Grover iteration (oracle + diffusion)
grover_oracle(qc_grover, secret)
grover_diffusion(qc_grover)

# Measure
qc_grover.measure([0, 1], [0, 1])

# Execute
job = execute(qc_grover, backend, shots=1024)
result = job.result()
counts = result.get_counts(qc_grover)

# Display results
console.print(f"\n[bold yellow]🔍 Searching for secret number: {secret} (binary: {format(secret, '02b')})[/bold yellow]\n")

# Find most common result
found = max(counts, key=counts.get)
found_decimal = int(found, 2)

if found_decimal == secret:
    console.print(f"[bold green]✅ SUCCESS! Found {found} (decimal: {found_decimal}) with {counts[found]} hits![/bold green]\n")
else:
    console.print(f"[bold red]❌ Found {found} (decimal: {found_decimal}) but secret was {secret}[/bold red]\n")

# Plot results
plt.figure(figsize=(10, 5))
plot_histogram(counts, figsize=(10, 5), color='lightcoral')
plt.title(f"Grover's Search Results (Secret: {format(secret, '02b')})", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

console.print(Panel(
    "[bold cyan]🎓 What just happened?[/bold cyan]\n\n"
    "• Classical search: Would need to try each number sequentially\n"
    "• Grover's algorithm: Finds the answer in √N steps\n"
    "• For 4 items: Classical=4 tries, Quantum=√4=2 tries\n"
    "• For 1,000,000 items: Classical=1M tries, Quantum=1000 tries!\n\n"
    "This is a [bold]quadratic speedup[/bold] - useful for breaking encryption!",
    border_style="cyan"
))

---

## 5️⃣ Interactive Widget: RSA Key Size Explorer

Let's explore how quantum computers threaten RSA encryption!

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# RSA key size selector
key_size_widget = widgets.Dropdown(
    options=[512, 1024, 2048, 3072, 4096],
    value=2048,
    description='RSA Key:',
    style={'description_width': '100px'}
)

output_widget = widgets.Output()

def analyze_rsa_security(key_size):
    """Analyze RSA security against quantum attacks"""
    with output_widget:
        clear_output(wait=True)
        
        # Estimate qubits needed for Shor's algorithm
        qubits_needed = key_size * 2
        
        # Classical vs Quantum complexity
        classical_time = 2 ** (key_size / 3)  # Simplified
        quantum_time = key_size ** 2  # Polynomial time
        
        # Security assessment
        if key_size <= 1024:
            status = "[bold red]❌ VULNERABLE[/bold red]"
            risk = "HIGH"
            color = "red"
        elif key_size <= 2048:
            status = "[bold yellow]⚠️  AT RISK[/bold yellow]"
            risk = "MEDIUM"
            color = "yellow"
        else:
            status = "[bold green]✅ CURRENT SAFE[/bold green]"
            risk = "LOW (but not future-proof)"
            color = "green"
        
        # Create results table
        table = Table(title=f"RSA-{key_size} Security Analysis", border_style=color)
        table.add_column("Metric", style="cyan")
        table.add_column("Value", style="magenta")
        
        table.add_row("Key Size", f"{key_size} bits")
        table.add_row("Qubits Needed", f"~{qubits_needed:,} logical qubits")
        table.add_row("Classical Time", "2^170 operations" if key_size >= 2048 else "2^80 operations")
        table.add_row("Quantum Time", f"~{quantum_time:,} operations")
        table.add_row("Risk Level", risk)
        
        console.print(table)
        console.print(f"\nStatus: {status}\n")
        
        # Recommendations
        if key_size <= 1024:
            msg = (
                "[bold red]🚨 CRITICAL:[/bold red] This key size can be broken TODAY\n"
                "with advanced hardware. Upgrade immediately!\n\n"
                "Recommended: RSA-3072+ or migrate to post-quantum crypto."
            )
        elif key_size == 2048:
            msg = (
                "[bold yellow]⚠️  WARNING:[/bold yellow] Current standard but vulnerable\n"
                "within 10-15 years to quantum computers.\n\n"
                "Consider: Planning migration to post-quantum algorithms."
            )
        else:
            msg = (
                "[bold green]✅ SECURE:[/bold green] Safe against current threats.\n\n"
                "But remember: Harvest Now, Decrypt Later attacks exist!\n"
                "Plan for post-quantum migration in next 5-10 years."
            )
        
        console.print(Panel(msg, border_style=color))

# Connect widget to function
widgets.interactive(analyze_rsa_security, key_size=key_size_widget)

# Display
display(widgets.VBox([
    widgets.HTML("<h3>🔐 RSA Security Analysis Against Quantum Attacks</h3>"),
    key_size_widget,
    output_widget
]))

# Run initial analysis
analyze_rsa_security(2048)

---

## 6️⃣ Explore More Tutorials

🎉 **Congratulations!** You've completed the quick start.

### 📚 Next Steps

Explore comprehensive tutorials in the `notebooks/` directory:

1. **[01-IBM_Quantum_Experience_Integration.ipynb](01-IBM_Quantum_Experience_Integration.ipynb)** - Connect to real quantum hardware
2. **[02-Shors_Algorithm_RSA_Exploitation.ipynb](02-Shors_Algorithm_RSA_Exploitation.ipynb)** - Factor RSA keys
3. **[03-Grovers_Algorithm_Symmetric_Key_Attacks.ipynb](03-Grovers_Algorithm_Symmetric_Key_Attacks.ipynb)** - Break AES/SSH
4. **[04-Quantum_Network_Scanning.ipynb](04-Quantum_Network_Scanning.ipynb)** - Find vulnerable systems
5. **[05-Harvest_Now_Decrypt_Later_Attacks.ipynb](05-Harvest_Now_Decrypt_Later_Attacks.ipynb)** - Future threats
6. **[06-Post_Quantum_Cryptography_Analysis.ipynb](06-Post_Quantum_Cryptography_Analysis.ipynb)** - PQC solutions
7. **[07-Quantum_Machine_Learning_Cryptanalysis.ipynb](07-Quantum_Machine_Learning_Cryptanalysis.ipynb)** - QML attacks
8. **[08-Houdinis_Advanced_Features.ipynb](08-Houdinis_Advanced_Features.ipynb)** - Framework deep dive
9. **[09-Houdinis_Framework_Conclusion.ipynb](09-Houdinis_Framework_Conclusion.ipynb)** - Summary & roadmap

### 🔗 Useful Links

- **Documentation:** https://maurorisonho.github.io/Houdinis/
- **GitHub:** https://github.com/maurorisonho/Houdinis
- **Install Locally:** `pip install git+https://github.com/maurorisonho/Houdinis`
- **Report Issues:** https://github.com/maurorisonho/Houdinis/issues

### 💡 Tips

- Save your work by downloading notebooks (File → Download)
- For production use, install Houdinis locally
- Join our community (see GitHub for links)
- Contribute tutorials and improvements!

---

## 🙏 Thank You!

Thanks for trying Houdinis! If you found this useful:
- ⭐ Star us on GitHub
- 📢 Share with colleagues
- 🐛 Report bugs
- 💡 Suggest features

**Happy quantum hacking!** 🔐⚛️

In [ ]:
# Final message
console.print(Panel.fit(
    "[bold green]🎉 Tutorial Complete![/bold green]\n\n"
    "You've learned:\n"
    "✅ Quantum circuit creation\n"
    "✅ Bell state entanglement\n"
    "✅ Grover's search algorithm\n"
    "✅ RSA security analysis\n\n"
    "[bold cyan]Next: Explore the notebooks/ directory![/bold cyan]",
    title="Success!",
    border_style="green"
))